In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install mxnet
!pip install gluonnlp pandas tqdm
!pip install sentencepiece
!pip install transformers==3.0.2
!pip install torch

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 49.1 MB 1.2 MB/s 
  Attempting uninstall: graphviz
    Found existing installation: graphviz 0.10.1
    Uninstalling graphviz-0.10.1:
      Successfully uninstalled graphviz-0.10.1
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 344 kB 4.7 MB/s 
  Created wheel for gluonnlp: filename=gluonnlp-0.10.0-cp37-cp37m-linux_x86_64.whl size=595737 sha256=137e1de0c329a27529d9c59b7cfd685f6cb289d610e13b1b87d1f0915fc2ec6f
  Stored in directory: /root/.cache/pip/wheels/be/b4/06/7f3fdfaf707e6b5e98b79c041e023acffbe395d78a527eae00
Successfully built gluonnlp
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.3 MB 4.7 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg

In [3]:
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://****@github.com/SKTBrain/KoBERT.git (to revision master) to /tmp/pip-req-build-5dvknrs5
  Running command git clone -q 'https://****@github.com/SKTBrain/KoBERT.git' /tmp/pip-req-build-5dvknrs5
     |████████████████████████████████| 129 kB 5.0 MB/s 
     |████████████████████████████████| 54.7 MB 19 kB/s 
     |████████████████████████████████| 4.5 MB 59.7 MB/s 
     |████████████████████████████████| 1.2 MB 54.4 MB/s 
     |██████████████████████████████▎ | 834.1 MB 1.2 MB/s eta 0:00:40tcmalloc: large alloc 1147494400 bytes == 0x39dc0000 @  0x7f643b0b7615 0x58e046 0x4f2e5e 0x4d19df 0x51b31c 0x5b41c5 0x58f49e 0x51b221 0x5b41c5 0x58f49e 0x51837f 0x4cfabb 0x517aa0 0x4cfabb 0x517aa0 0x4cfabb 0x517aa0 0x4ba70a 0x538136 0x590055 0x51b180 0x5b41c5 0x58f49e 0x51837f 0x5b41c5 0x58f49e 0x51740e 0x58f2a7 0x517947 0x5b41c5 0x58f49e
     |████████████████████████████████| 881.9 MB 1

In [4]:
!pip uninstall folium 
!pip install folium==0.2.1

Found existing installation: folium 0.12.1.post1
Uninstalling folium-0.12.1.post1:
  Would remove:
    /usr/local/lib/python3.7/dist-packages/folium-0.12.1.post1.dist-info/*
    /usr/local/lib/python3.7/dist-packages/folium/*
Proceed (y/n)? y
  Successfully uninstalled folium-0.12.1.post1
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 69 kB 3.6 MB/s 
  Created wheel for folium: filename=folium-0.2.1-py3-none-any.whl size=79808 sha256=22f806b3afb4db0c51c9f217f6f6ae65fb7ae33f583176c3eab172b052bf68c2
  Stored in directory: /root/.cache/pip/wheels/9a/f0/3a/3f79a6914ff5affaf50cabad60c9f4d565283283c97f0bdccf
Successfully built folium
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datascience 0.17.5 requires folium>=0.9.1, but you have folium 0.2.1 which is incompatible.


In [11]:
import torch
import pandas as pd
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook
from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

In [8]:
device = torch.device("cuda:0")

In [9]:
bertmodel, vocab = get_pytorch_kobert_model()

/content/.cache/kobert_v1.zip[██████████████████████████████████████████████████]
/content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece[██████████████████████████████████████████████████]


In [13]:
# dataset은 반드시 TSV 파일이어야 한다. 
# field_indices = [학습시킬 데이터의 인덱스, 데이터 레이블의 인덱스]
# num_discard_samples = 데이터의 상단에서 제외할 row의 개수 [ default = 0 ]

dataset_train = nlp.data.TSVDataset('/content/reset_star_train.tsv', field_indices=[0,1], num_discard_samples=1)  
dataset_test = nlp.data.TSVDataset('/content/reset_star_test.tsv', field_indices=[0,1], num_discard_samples=1)

In [14]:
dataset_train[0]

['척테일러 같은 색상 같은 사이즈 세번째 구매인데 제가 알던 척테일러가 아니에요... 끈은 짧고 코는 크고 발은 불편하고...', '1']

In [15]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

In [16]:
max_len = 64
batch_size = 64
warmup_ratio = 0.1
num_epochs = 10
max_grad_norm = 1  
log_interval = 200
learning_rate =  5e-5

In [17]:
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)
#BERTDataset 클래스 이용, TensorDataset으로 만들어주기
data_train = BERTDataset(dataset_train, 0, 1, tok, max_len, True, False)
data_test = BERTDataset(dataset_test, 0, 1, tok, max_len, True, False)

using cached model. /content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [18]:
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=4)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=4)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [19]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=5, ##주의: 클래스 수 바꾸어 주세요!##
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [20]:
model = BERTClassifier(bertmodel, dr_rate=0.5).to(device)

In [21]:
# Prepare optimizer and schedule (linear warmup and decay)
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

In [22]:
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

In [23]:
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} validation acc {}".format(e+1, test_acc / (batch_id+1)))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """


  0%|          | 0/25 [00:00<?, ?it/s]

epoch 1 batch id 1 loss 1.6749995946884155 train acc 0.265625
epoch 1 train acc 0.211875


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:22: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


  0%|          | 0/7 [00:00<?, ?it/s]

...

epoch 9 validation acc 0.4486607142857143


  0%|          | 0/25 [00:00<?, ?it/s]

epoch 10 batch id 1 loss 0.6538413763046265 train acc 0.78125
epoch 10 train acc 0.78875


  0%|          | 0/7 [00:00<?, ?it/s]

epoch 10 validation acc 0.44642857142857145


# TEST

In [24]:
new_test = nlp.data.TSVDataset('/content/reset_star_test.tsv', field_indices=[0,1], num_discard_samples=1)
test_set = BERTDataset(new_test , 0, 1, tok, max_len, True, False)
test_input = torch.utils.data.DataLoader(test_set, batch_size=1, num_workers=4)

In [25]:
for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_input)): 
  token_ids = token_ids.long().to(device) 
  segment_ids = segment_ids.long().to(device) 
  valid_length= valid_length 
  out = model(token_ids, valid_length, segment_ids)
  prediction = out.cpu().detach().numpy().argmax()
  print(batch_id, prediction)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """Entry point for launching an IPython kernel.


  0%|          | 0/400 [00:00<?, ?it/s]

0 0
1 0
2 1
3 1
4 4
5 4
6 3
7 4
8 3
9 1
10 3
11 2
12 0
13 1
14 2
15 4
16 4
17 4
18 4
19 1
20 4
21 4
22 0
23 1
24 2
25 1
26 2
27 1
28 1
29 1
30 0
31 2
32 0
33 2
34 0
35 1
36 3
37 3
38 0
39 2
40 4
41 0
42 0
43 1
44 0
45 2
46 3
47 4
48 4
49 0
50 1
51 1
52 1
53 1
54 4
55 2
56 3
57 0
58 3
59 2
60 2
61 0
62 4
63 2
64 4
65 1
66 2
67 0
68 1
69 0
70 2
71 3
72 4
73 4
74 4
75 4
76 3
77 2
78 2
79 0
80 4
81 1
82 4
83 2
84 1
85 4
86 4
87 4
88 0
89 3
90 1
91 1
92 2
93 1
94 4
95 2
96 1
97 0
98 1
99 2
100 3
101 3
102 2
103 2
104 2
105 4
106 1
107 4
108 1
109 3
110 4
111 2
112 0
113 4
114 3
115 2
116 0
117 0
118 3
119 0
120 2
121 2
122 4
123 1
124 1
125 0
126 4
127 3
128 0
129 0
130 3
131 4
132 0
133 2
134 4
135 2
136 4
137 1
138 2
139 1
140 2
141 4
142 3
143 1
144 0
145 3
146 3
147 2
148 2
149 4
150 4
151 4
152 3
153 4
154 1
155 3
156 4
157 3
158 3
159 1
160 4
161 3
162 2
163 0
164 2
165 3
166 2
167 3
168 1
169 1
170 2
171 0
172 2
173 3
174 2
175 3
176 3
177 1
178 4
179 1
180 3
181 1
182 1
183 1
184 3


In [26]:
def test_calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    test_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return test_acc

In [27]:
for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_input)): 
    token_ids = token_ids.long().to(device) 
    segment_ids = segment_ids.long().to(device) 
    valid_length= valid_length 
    label = label.long().to(device)
    out = model(token_ids, valid_length, segment_ids)
    prediction = out.cpu().detach().numpy().argmax()
    test_acc += calc_accuracy(out, label)
    print(batch_id, prediction+1)
print("epoch {} validation acc {}".format(e+1, test_acc / (batch_id+1)))


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """Entry point for launching an IPython kernel.


  0%|          | 0/400 [00:00<?, ?it/s]

0 1
1 1
2 2
3 2
4 5
5 5
6 4
7 5
8 4
9 2
10 4
11 3
12 1
13 2
14 3
15 5
16 5
17 5
18 5
19 2
20 5
21 5
22 1
23 2
24 3
25 2
26 3
27 2
28 2
29 2
30 1
31 3
32 1
33 3
34 1
35 2
36 4
37 4
38 1
39 3
40 5
41 1
42 1
43 2
44 1
45 3
46 4
47 5
48 5
49 1
50 2
51 2
52 2
53 2
54 5
55 3
56 4
57 1
58 4
59 3
60 3
61 1
62 5
63 3
64 5
65 2
66 3
67 1
68 2
69 1
70 3
71 4
72 5
73 5
74 5
75 5
76 4
77 3
78 3
79 1
80 5
81 2
82 5
83 3
84 2
85 5
86 5
87 5
88 1
89 4
90 2
91 2
92 3
93 2
94 5
95 3
96 2
97 1
98 2
99 3
100 4
101 4
102 3
103 3
104 3
105 5
106 2
107 5
108 2
109 4
110 5
111 3
112 1
113 5
114 4
115 3
116 1
117 1
118 4
119 1
120 3
121 3
122 5
123 2
124 2
125 1
126 5
127 4
128 1
129 1
130 4
131 5
132 1
133 3
134 5
135 3
136 5
137 2
138 3
139 2
140 3
141 5
142 4
143 2
144 1
145 4
146 4
147 3
148 3
149 5
150 5
151 5
152 4
153 5
154 2
155 4
156 5
157 4
158 4
159 2
160 5
161 4
162 3
163 1
164 3
165 4
166 3
167 4
168 2
169 2
170 3
171 1
172 3
173 4
174 3
175 4
176 4
177 2
178 5
179 2
180 4
181 2
182 2
183 2
184 4


In [28]:
import warnings
warnings.filterwarnings('ignore')

In [29]:
# tokenizer = get_tokenizer()
# tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)



def predict(predict_sentence):

    data = [predict_sentence, '0']
    dataset_another = [data]

    another_test = BERTDataset(dataset_another, 0, 1, tok, max_len, True, False)
    test_dataloader = torch.utils.data.DataLoader(another_test, batch_size=batch_size, num_workers=5)
    
    model.eval()

    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(test_dataloader):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)

        valid_length= valid_length
        label = label.long().to(device)

        out = model(token_ids, valid_length, segment_ids)


        test_eval=[]
        for i in out:
            logits=i
            logits = logits.detach().cpu().numpy()

            if np.argmax(logits) == 0:
                test_eval.append("1점")
            elif np.argmax(logits) == 1:
                test_eval.append("2점")
            elif np.argmax(logits) == 2:
                test_eval.append("3점")
            elif np.argmax(logits) == 3:
                test_eval.append("4점")
            elif np.argmax(logits) == 4:
                test_eval.append("5점")
            

        print(test_eval[0] + " 입니다.")

In [30]:
predict(input('>> 리뷰 내용을 입력해주세요 : '))


>> 리뷰 내용을 입력해주세요 : 중국산 아니랄까봐... 저렴한만큼 합니다. 손잡이 마무리 엉망이에요... 이도 안맞게 박음질처리를 했네요....
1점 입니다.
